# [Module 4.1] Deploy Built-in TFS Image


여기서는 다음과 같은 작업을 합니다.

- 모델 아티펙트 (model.tar.gz) 파일을 S3에서 로컬에 다운로드
- TF Saved_Model 의 정의를 확인
- SageMaker Model 생성
- Endpoint 생성
- Inference의 Request Serializer and Deserializer 생성
- 프리딕터 생성
- 셈플 데이타로 추론

---
이 노트북은 약 10분 정도 소요 됩니다.


필요한 프로그램 설치

In [1]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0
!pip install -q sagemaker==1.56.1

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

astroid 2.3.3 requires wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [2]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Download the Model to the Notebook

In [3]:
%store -r training_job_name

In [4]:
print(training_job_name)

tensorflow-training-2020-08-16-01-08-59-496


In [5]:
model_download = 'model'
os.makedirs(model_download, exist_ok=True)

In [6]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

download: s3://sagemaker-ap-northeast-2-343441690612/tensorflow-training-2020-08-16-01-08-59-496/output/model.tar.gz to model/model.tar.gz


In [7]:
# !tar -xvzf   {model_download}/model.tar.gz
# !saved_model_cli show --all --dir ./tensorflow/saved_model/0/

## SageMaker Model 생성

In [8]:
import os
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

## Endpoint 생성

In [9]:
instance_type='ml.m4.xlarge'

deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)



-------------!

In [10]:
endpoint_name = deployed_model.endpoint
print('Endpoint name:  {}'.format(endpoint_name))

Endpoint name:  tensorflow-inference-2020-08-16-01-43-10-321


## Inference Request Serializer and Deserializer 생성

In [11]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)
    
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

#        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
#         for log_probability in log_probabilities:
#             softmax = tf.nn.softmax(log_probability)    
#             predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
#             predicted_class = self.classes[predicted_class_idx]
#             predicted_classes.append(predicted_class)

        return log_probabilities    

## Predictor 생성

In [12]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=32)

response_handler = ResponseHandler(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

## Inference 실행

In [13]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [14]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2824,yo this procrastination is real i didn't go o...,7
8830,overnight kila bes merlengggg,0
1969,x x me we smacked this bih we jus getting sta...,2
5410,yes you did ny is your home and you should be...,1
1604,his jersey sleeves is the reason c'mon everyon...,3
2338,one day promise intime myart,4
2970,too much luv for tuna pasta n sweetcorn,1
936,going from waking up at 3 pm everyday to waki...,6
9360,which,9
748,love it sis,0


In [15]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


In [16]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 3
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
#     print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
#    predicted_classes = predictor.predict(reviews)    
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2])                                       
        ))    
        



tweet:  yo this procrastination is real i didn't go out all weekend and have complete zero homework i have a problem 
Ground_truth- 7:😭
 
Prediction: 8,🙄,9,🤔,3,😂 
 
tweet: overnight kila bes merlengggg 
Ground_truth- 0:❤
 
Prediction: 6,😩,3,😂,5,😍 
 
tweet:  x x me we smacked this bih we jus getting started 
Ground_truth- 2:🔥
 
Prediction: 6,😩,3,😂,2,🔥 
 
tweet:  yes you did ny is your home and you should be there no where else ms sherri 
Ground_truth- 1:💕
 
Prediction: 1,💕,4,😊,3,😂 
 
tweet: his jersey sleeves is the reason c'mon everyone knows that 
Ground_truth- 3:😂
 
Prediction: 6,😩,3,😂,0,❤ 
 
tweet:  one day promise intime myart 
Ground_truth- 4:😊
 
Prediction: 8,🙄,7,😭,9,🤔 
 
tweet: too much luv for tuna pasta n sweetcorn 
Ground_truth- 1:💕
 
Prediction: 6,😩,5,😍,8,🙄 
 
tweet:  going from waking up at 3 pm everyday to waking up at 6 am 
Ground_truth- 6:😩
 
Prediction: 8,🙄,9,🤔,7,😭 
 
tweet:  which 
Ground_truth- 9:🤔
 
Prediction: 0,❤,1,💕,9,🤔 
 
tweet: love it sis 
Ground_truth- 0:❤
 
P

## Past Version

In [17]:
import tensorflow as tf
import json

sample_df = test_df.sample(10)
columns = ['TWEET', 'LABEL']
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]

    predicted_classes = predictor.predict(reviews)
    print(predicted_classes)
        



[[-0.400611103, -1.16453958, -1.89889371, 4.70898628, -2.37440658, -1.60534883, 3.95935464, -1.55257404, -1.79060662, -1.94558775]]
[[-1.20432031, -1.82773113, -0.391512632, 0.703009248, -1.59350145, -0.565404058, 7.18686295, -1.97893572, -1.2113502, -1.69550371]]
[[-1.83954346, -2.39527082, 1.8454771, -0.571332097, -2.09438753, 0.175423399, 5.93926525, -2.12616, 0.507222772, -1.90001047]]
[[-1.51556039, 1.99300051, -2.42727852, 4.90277386, -1.01229858, -1.762568, 0.475168973, -1.92735767, -1.88486302, -1.84640229]]
[[-0.78829211, -1.94221628, -0.810397327, 2.82171583, -2.31329346, -2.84787273, 2.85517621, -1.29353356, 1.69144058, -1.90860903]]
[[-2.38983226, -1.91434097, -1.83881164, -0.366253287, -2.17718744, -2.49873948, 0.49742192, -0.975801587, 7.15242624, -0.499093056]]
[[-0.329123735, -1.50071621, -1.88950539, 1.39940095, -0.332201302, 5.98732853, -1.2794199, -1.39521348, -2.3023181, -1.70894337]]
[[-1.21936142, -0.279880226, -1.39440358, 1.44727635, -2.01546574, -1.44794595, 6.

In [18]:
import tensorflow as tf
import json

sample_df = test_df.sample(10)
columns = ['TWEET', 'LABEL']
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]

    predicted_classes = predictor.predict(reviews)[0]
    print('Prediction, Ground_truth- {}:{}, {}:{} \n tweet: {}'.format(
        predicted_classes, 
        tweet_util.get_emo_class_label(predicted_classes),
        label, 
        tweet_util.get_emo_class_label(label),        
        tweet))    

        



Prediction, Ground_truth- [3.77719784, 3.43111205, -1.69310737, -2.02781153, -1.97104251, -1.358307, 1.63716865, -2.22402334, -2.48495936, -1.99037027]:None, 4:😊 
 tweet:  thank you i am so happy 
Prediction, Ground_truth- [-1.0546068, 3.20887256, -3.05958223, 0.414651603, -1.70333171, -1.96098542, 1.28481448, 1.59856355, -2.34978557, -1.74867749]:None, 6:😩 
 tweet:  i will never understand how a friendship like kalin and myles just ends
Prediction, Ground_truth- [0.0448568128, -0.307889551, -2.46420431, -0.81351316, 0.686781883, 6.1663456, -2.12994337, -0.655496597, -2.31159425, -1.73348486]:None, 5:😍 
 tweet: mrs sonakshidixit is looking so beautiful devakshikishaadi krpkab
Prediction, Ground_truth- [0.607466161, 0.0160209909, -1.20571065, -1.29471827, -1.30522609, -2.12841, -0.0549491271, -1.85945106, -1.2868154, 3.62702799]:None, 4:😊 
 tweet:  start deleting career's on his games start deleting career's on his games start deleting career's on his games
Prediction, Ground_truth- [-0